In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import numpy as np


In [6]:
# Librerías
import pandas as pd
import numpy as np


movies_path = "ml-1m/movies.dat"
ratings_path = "ml-1m/ratings.dat"
users_path = "ml-1m/users.dat"


# Carga de los datos
users = pd.read_csv(users_path, sep="::", engine="python", names=["UserID", "Gender", "Age", "Occupation", "Zip-code"], encoding="latin-1")
movies = pd.read_csv(movies_path, sep="::", engine="python", names=["MovieID", "Title", "Genres"], encoding="latin-1")
ratings = pd.read_csv(ratings_path, sep="::", engine="python", names=["UserID", "MovieID", "Rating", "Timestamp"], encoding="latin-1")

# Mostrar primeras filas
ratings.head()



,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
from sklearn.preprocessing import LabelEncoder

user_enc = LabelEncoder()
movie_enc = LabelEncoder()

ratings['user'] = user_enc.fit_transform(ratings['UserID'])
ratings['movie'] = movie_enc.fit_transform(ratings['MovieID'])

num_users = ratings['user'].nunique()
num_movies = ratings['movie'].nunique()


In [8]:
from collections import defaultdict

max_seq_len = 10  # longitud fija de historial

user_histories = defaultdict(list)
sequence_data = []

# Ordenamos por timestamp
ratings = ratings.sort_values(by=['user', 'Timestamp'])

# Generar secuencias
for row in ratings.itertuples():
    u, m = row.user, row.movie
    hist = user_histories[u][-max_seq_len:]
    if len(hist) >= 1:  # solo si hay al menos un ítem en historial
        padded_hist = [0] * (max_seq_len - len(hist)) + hist  # pad left
        sequence_data.append((padded_hist, m))
    user_histories[u].append(m)


In [9]:
import random

class GRU4RecNegDataset(Dataset):
    def __init__(self, sequence_data, num_items, num_negatives=20):
        self.sequence_data = sequence_data
        self.num_items = num_items
        self.num_negatives = num_negatives

    def __len__(self):
        return len(self.sequence_data)

    def __getitem__(self, idx):
        seq, pos_item = self.sequence_data[idx]
        neg_items = []
        while len(neg_items) < self.num_negatives:
            neg = random.randint(1, self.num_items - 1)  # evitar 0 (padding)
            if neg != pos_item:
                neg_items.append(neg)

        return (
            torch.tensor(seq, dtype=torch.long),                    # input_seq
            torch.tensor(pos_item, dtype=torch.long),              # positivo
            torch.tensor(neg_items, dtype=torch.long)              # negativos
        )


In [10]:
from torch.utils.data import random_split, DataLoader

neg_dataset = GRU4RecNegDataset(sequence_data, num_items=num_movies, num_negatives=20)

train_size = int(0.8 * len(neg_dataset))
val_size = len(neg_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(neg_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256)


In [11]:
class GRU4RecRankingModel(nn.Module):
    def __init__(self, num_items, embedding_dim=64, hidden_dim=128, num_layers=1, dropout=0.3):
        super(GRU4RecRankingModel, self).__init__()
        self.embedding = nn.Embedding(num_items, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.item_proj = nn.Embedding(num_items, hidden_dim)  # para comparar con salida de GRU

    def forward(self, input_seq, pos_items, neg_items):
        # input_seq: (B, seq_len), pos_items: (B,), neg_items: (B, N)
        emb = self.embedding(input_seq)                         # (B, seq_len, emb_dim)
        gru_out, _ = self.gru(emb)                              # (B, seq_len, hidden_dim)
        user_emb = gru_out[:, -1, :]                            # (B, hidden_dim)

        # Positivo
        pos_emb = self.item_proj(pos_items)                     # (B, hidden_dim)
        pos_scores = torch.sum(user_emb * pos_emb, dim=1)       # (B,)

        # Negativos
        neg_emb = self.item_proj(neg_items)                     # (B, N, hidden_dim)
        user_emb_exp = user_emb.unsqueeze(1).expand_as(neg_emb)  # (B, N, hidden_dim)
        neg_scores = torch.sum(user_emb_exp * neg_emb, dim=2)   # (B, N)

        return pos_scores, neg_scores


In [12]:
def train_gru4rec_ranking(model, train_loader, val_loader, epochs=10, lr=0.001, device='cpu'):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for input_seq, pos_item, neg_items in train_loader:
            input_seq = input_seq.to(device)
            pos_item = pos_item.to(device)
            neg_items = neg_items.to(device)

            optimizer.zero_grad()
            pos_scores, neg_scores = model(input_seq, pos_item, neg_items)

            # Construir etiquetas: 1 para positivo, 0 para negativos
            target_pos = torch.ones_like(pos_scores)
            target_neg = torch.zeros_like(neg_scores)

            # Calcular BCE
            loss_pos = criterion(pos_scores, target_pos)
            loss_neg = criterion(neg_scores, target_neg)
            loss = loss_pos + loss_neg.mean()

            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}: Train Loss = {total_loss/len(train_loader):.4f}")


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
ranking_model = GRU4RecRankingModel(num_items=num_movies, embedding_dim=64, hidden_dim=128).to(device)

train_gru4rec_ranking(
    model=ranking_model,
    train_loader=train_loader,
    val_loader=val_loader,  # aún no la usamos, pero podemos extender para Recall@10 luego
    epochs=30,
    lr=0.001,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)


/home/hjasi/Documentos/DeepLearning/.venv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


Epoch 1: Train Loss = 1.0950
Epoch 2: Train Loss = 0.8126
Epoch 3: Train Loss = 0.7095
Epoch 4: Train Loss = 0.6522
Epoch 5: Train Loss = 0.6134
Epoch 6: Train Loss = 0.5850
Epoch 7: Train Loss = 0.5632
Epoch 8: Train Loss = 0.5448
Epoch 9: Train Loss = 0.5289
Epoch 10: Train Loss = 0.5150
Epoch 11: Train Loss = 0.5018
Epoch 12: Train Loss = 0.4904
Epoch 13: Train Loss = 0.4794
Epoch 14: Train Loss = 0.4694
Epoch 15: Train Loss = 0.4596
Epoch 16: Train Loss = 0.4505
Epoch 17: Train Loss = 0.4417
Epoch 18: Train Loss = 0.4336
Epoch 19: Train Loss = 0.4260
Epoch 20: Train Loss = 0.4184
Epoch 21: Train Loss = 0.4113
Epoch 22: Train Loss = 0.4046
Epoch 23: Train Loss = 0.3979
Epoch 24: Train Loss = 0.3922
Epoch 25: Train Loss = 0.3858
Epoch 26: Train Loss = 0.3804
Epoch 27: Train Loss = 0.3745
Epoch 28: Train Loss = 0.3695
Epoch 29: Train Loss = 0.3648
Epoch 30: Train Loss = 0.3601


In [17]:
def evaluate_gru4rec_recall_at_k(model, val_loader, k=10, device='cpu'):
    model.eval()
    model.to(device)
    hits = 0
    total = 0

    with torch.no_grad():
        for input_seq, pos_item, _ in val_loader:
            input_seq = input_seq.to(device)
            pos_item = pos_item.to(device)

            # Puntuaciones para todos los ítems
            emb = model.embedding(input_seq)                # (B, L, emb_dim)
            gru_out, _ = model.gru(emb)                     # (B, L, hidden_dim)
            user_emb = gru_out[:, -1, :]                    # (B, hidden_dim)

            all_items = torch.arange(model.item_proj.num_embeddings).to(device)
            item_emb = model.item_proj(all_items)           # (num_items, hidden_dim)

            scores = torch.matmul(user_emb, item_emb.T)     # (B, num_items)
            top_k = torch.topk(scores, k=k, dim=1).indices  # (B, K)

            for i in range(pos_item.size(0)):
                if pos_item[i] in top_k[i]:
                    hits += 1
                total += 1

    recall_at_k = hits / total
    print(f"Recall@{k}: {recall_at_k:.4f}")
    return recall_at_k


In [19]:
evaluate_gru4rec_recall_at_k(model=ranking_model, val_loader=val_loader, k=10, device=device)


Recall@10: 0.1473


0.14730378104348352